<a href="https://colab.research.google.com/github/francji1/01NAEX/blob/main/R/01NAEX_Exercise_06_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

	
# O1NAEX Exercise 06

In [ ]:
cat(system('sudo apt-get install -y gmp', intern=TRUE), sep = "\n")
cat(system('sudo apt-get install -y partitions', intern=TRUE), sep = "\n")
cat(system('sudo apt-get install -y DoE.base', intern=TRUE), sep = "\n")

list_of_packages <- c("tidyverse","MASS","FrF2", "lattice") # "agricolae","nortest","lattice","pwr",
missing_packages <- list_of_packages[!(list_of_packages %in% installed.packages()[,"Package"])]
missing_packages
if(length(missing_packages)) install.packages(missing_packages)
lapply(list_of_packages, library, character.only = TRUE)


In [ ]:
tibble::tibble(
  Package = names(installed.packages()[,3]),
  Version = unname(installed.packages()[,3])
)

##	Problem 6.31 + 6.32
from the chapter 6, D. C. Montgomery DAoE - 8. edition.
	
An experiment was conducted on a chemical process
that produces a polymer. The four factors studied were temperature
(A), catalyst concentration (B), time (C), and pressure
(D). Two responses, molecular weight and viscosity, were
observed. The design matrix and response data are following:

In [ ]:
df631 <- read.table("https://raw.githubusercontent.com/francji1/01NAEX/main/data/Problem_6_31.txt",header=TRUE,sep=";")
df631
print("First 16 measurments:")
df631_first_16 <- df631[1:16,]
df631_first_16

Use only first sixteen measurements, center points will be covered in the following lesson.

## Tasks:
1. Consider only the molecular weight response. Plot the
effect estimates on a normal probability scale. What
effects appear important?
2. Use an analysis of variance to confirm the results from the first 
part.
3. Write down a regression model to predict molecular
weight as a function of the important variables.
4. Analyze the residuals and comment on model adequacy.
5. Repeat parts 1.–4. using the viscosity response.

Use the regression models for molecular weight and viscosity to answer the following questions.
6. Construct a response surface contour plot for molecular
weight. In what direction would you adjust therocess variables to increase molecular weight?
7. Construct a response surface contour plot for viscosity.
In what direction would you adjust the process variables
to decrease viscosity?
8. What operating conditions would you recommend if it
was necessary to produce a product with molecular
weight between 2400 and 2500 and the lowest possible
viscosity?



In [ ]:
m         <-  FrF2(2^4, 4, replications = 1, randomize = F,factor.names = c("A", "B","C", "D"))
Weight    <- df631_first_16[1:16,"Weight"]
#Viscosity <- df631[1:16,"Viscosity"]
m         <- add.response(m,Weight)
#m         <- add.response(m,Viscosity)
response.names(m)
m

In [ ]:
DanielPlot(m,code=TRUE)
# Classical effects qqplot
qqplot(DanielPlot(m,alpha=0.1)$x,DanielPlot(m)$y)
qqline(DanielPlot(m,alpha=0.1)$y)

# half normal plot of effects
DanielPlot(m,code=TRUE,alpha=0.1,half=TRUE)


# Weight as a response variable

## Task 1
Consider only the molecular weight response. Plot the
effect estimates on a normal probability scale. What
effects appear important.

In [ ]:
par(mfrow = c(1,2))
options(repr.plot.width = 20, repr.plot.height = 10)

model_weight <- lm(Weight~A*B*C*D, data= m)
summary(model_weight)
barplot(sort(model_weight$coeff[2:(2^4-1)]), las=2)
qqnorm(model_weight$coeff[2:(2^4-1)], cex=1.3, pch=15)
qqline(model_weight$coeff[2:(2^4-1)], cex=1.3, pch=15)

Normal qqplot of coeffitients shows that groups A, C and interaction between groups A:B is significant. 

## Task 2
Use analysis of variance to confirm the results from the first 
part.

In [ ]:
anova(aov(Weight ~ A*B*C*D, data=m))

In [ ]:
anova(aov(Weight ~ (.)^2, data=m))

In [ ]:
anova(aov(Weight ~ A*B + C, data=m))

Because we have 1 replication of the experiment, there are 0 degrees of freedom for error estimate. That is why Residuals are equal to 0 and NaN values in F-value and p-value. 
Nonetheless, by looking at the sum of square values, it can be seen that the effects of groups A, C, and interaction AB are significant.
This is in line with the results from the Q-Q plot of coefficients in Task 1.

## Task 3
Write down a regression model to predict molecular
weight as a function of the important variables.

In [ ]:
model_weight_only_important <- lm(Weight~A+C+B+A:B, data=m)
summary(model_weight_only_important)

I chose A, C, A:B as important categories based on the Q-Q plot and Anova test results. Variable B is kept in accordance with principles of construction of linear regression model i.e. Regressor B must be present when interaction A:B is present in the model. The results are in accordance with results from the Q-Q plot and Anova test. 

## Task 4
Analyze the residuals and comment on model adequacy.

In [ ]:
par(mfrow = c(3,2))
options(repr.plot.width = 15, repr.plot.height = 15)

plot(model_weight_only_important, which=c(1,2,3,4,5))

Q-Q plot indicates that residuals are normally distributed apart from observation 7, which deviates from the rest. This can be seen in all of the plots.

Also, observation has high leverage and seems to cause deviation from a straight line in the Residual vs. fitted fence.

Overall performance is good, and there is no visible pattern in residual plots, for this reason, I'm not going to remove the suspicious observations.

$R^{2}$ indicates good performance and in this model is a good indicator of performance as there are not that many regressors ($R^{2}$ can be artificially increased by adding additional regressors) and we only have categorical variables as regressors so possible high values of $R^{2}$ because of a large scale of regressors is not an issue.

# Viscosity as a response variable

## Task 1
Consider only the molecular viscosity response. Plot the
effect estimates on a normal probability scale. What
effects appear important.

In [ ]:
par(mfrow = c(1,2))
options(repr.plot.width = 20, repr.plot.height = 10)

model_viscos <- lm(Viscosity~A*B*C*D, data= m)
summary(model_viscos)
barplot(sort(model_viscos$coeff[2:(2^4-1)]), las=2)
qqnorm(model_viscos$coeff[2:(2^4-1)], cex=1.3, pch=15)
qqline(model_viscos$coeff[2:(2^4-1)], cex=1.3, pch=15)

Normal qqplot of coeffitients shows that groups A, B are significant. 

## Task 2
Use analysis of variance to confirm the results from the first 
part.

In [ ]:
anova(aov(Viscosity ~ A*B*C*D, data=m))

Because we have 1 replication of the experiment, there are 0 degrees of freedom for error estimate. That is why Residuals are equal to 0 and NaN values in F-value and p-value. 
Nonetheless, by looking at the sum of square values, it can be seen that the effects of groups A, B.
This is in line with the results from the Q-Q plot of coefficients in Task 1.

In [ ]:
anova(aov(Viscosity ~ A*B, data=m))

## Task 3
Write down a regression model to predict molecular
viscosity as a function of the important variables.

In [ ]:
model_viscos_only_important <- lm(Viscosity~A+B, data=m)
summary(model_viscos_only_important)

I chose A, B as important categories based on the Q-Q plot and Anova test results. The results are in accordance with results from the Q-Q plot and Anova test.

## Task 4
Analyze the residuals and comment on model_viscos adequacy.

In [ ]:
par(mfrow = c(3,2))
options(repr.plot.width = 15, repr.plot.height = 15)

plot(model_viscos_only_important, which=c(1,2,3,4,5))

Q-Q plot indicates that residuals are normally distributed apart from observation 12, which deviates from the rest. This can be seen in all of the plots.

Overall performance is not very good, there is a visible pattern in Residuals vs. fitted values, residual values increase whith fitted values. Also Scale-Location plot indicates bad performance. 

$R^{2}$ indicates good performance and in this model is a good indicator of performance as there are not that many regressors ($R^{2}$ can be artificially increased by adding additional regressors) and we only have categorical variables as regressors so possible high values of $R^{2}$ because of a large scale of regressors is not an issue.

Use the regression models for molecular weight and viscosity to answer the following questions.
## Task 6
Construct a response surface contour plot for molecular
weight. In what direction would you adjust therocess variables to increase molecular 

### Weight

In [ ]:
m$A_num <- df631_first_16$Temperature
m$B_num <- df631_first_16$Concentration
m$C_num <- df631_first_16$Time

weight_lm <- lm(Weight ~ I((A_num-110)/10) + I((B_num-6)/2) + I((C_num-25)/5) + I((A_num-110)/10):I((B_num-6)/2), data=m)

tmp <- data.frame(A_num=seq(100,120,by=0.5), B_num=seq(4,8,by=0.1), C_num=seq(20,30,by=0.25))
new_data <- expand.grid(tmp)
new_data$fit_weight <- predict(weight_lm,new_data)

wireframe(fit_weight ~ A_num * C_num, data = new_data ,scales=list(arrows=FALSE), drape=TRUE, perspective=TRUE, xlab="Temperature", ylab="Time", zlab="Weight", main="Response surface plot for molecular weight")

To increase weight, I would increase temperature and time.

## Task 7

### Viscosity

In [ ]:
viscos_lm <- lm(Viscosity ~ I((A_num-110)/10) + I((B_num-6)/2), data=m)
new_data$fit_viscosity <- predict(viscos_lm,new_data)

wireframe(fit_viscosity ~ A_num * B_num, data = new_data ,scales=list(arrows=FALSE), drape=TRUE, perspective=TRUE, xlab="Temperature", ylab="Concentration", zlab="Viscosity", main="Response surface plot for molecular viscosity")


To increase viscosity, I would increase temperature and concentration.

## Task 8
What operating conditions would you recommend if it was necessary to produce a product with molecular weight between 2400 and 2500 and the lowest possible viscosity?

In [ ]:
par(mfrow = c(1,3))
options(repr.plot.width = 10, repr.plot.height = 10)

contourplot(fit_weight~A_num*C_num, data=new_data, xlab="Temperature", ylab="Time" ,main="Dependence of fitted weight values on Temperature and Time")
contourplot(fit_weight~B_num*C_num, data=new_data, xlab="Concentration", ylab="Time", main="Dependence of fitted weight values on Concentration and Time")
contourplot(fit_viscosity~A_num*B_num, data=new_data, xlab="Temperature", ylab="Concentration", main="Dependence of fitted viscosity values on Temperature and Concentration")

I would recommend temperature of 110 with concentration under 6, the lower the better and with the length of time bigger than 26. Viscosity is then around 1400-1450.
The effect of pressure was not chosen in the final model as the effect was not prominent enough, nonetheless, it has a positive value of the regression parameter in the full model of molecular weight and a negative value of the regression parameter in the full model for molecular viscosity. I would opt for a higher value of pressure (75) as this would decrease viscosity and increase weight.

## Solution with fancy contour plots

In [ ]:
######################
# add more requirements
list_of_packages <- c("scatterplot3d","DoE.base","metR") #,"alr3","car","nortest","lattice","pwr","MASS",  "agricolae",
missing_packages <- list_of_packages[!(list_of_packages %in% installed.packages()[,"Package"])]
if(length(missing_packages)) install.packages(missing_packages)
lapply(list_of_packages, library, character.only = TRUE)

In [ ]:
m         <- FrF2(2^4, 4, replications = 1, randomize = F,
                  factor.names = c("A", "B","C", "D"))
Weight    <- df631[1:16,"Weight"]
Viscosity <- df631[1:16,"Viscosity"]
m         <- add.response(m,Weight)
m         <- add.response(m,Viscosity)

response.names(m)
print(m)


In [ ]:
# Weight
MEPlot(m, response = "Weight")
IAPlot(m, response = "Weight")
# vypada to, ze A a C budou mit vliv
# z interakci A:B

summary(aov(Weight ~ A*B*C*D, data=m))

model00 = lm(Weight~A*B*C*D,  data=m)
barplot(sort(model00$coeff[2:(2^4-1)]))
qqnorm(model00$coeff[2:(2^4-1)], cex = 1.3, pch = 15)
qqline(model00$coeff[2:(2^4-1)], cex = 1.3, pch = 15)



In [ ]:
DanielPlot(m,code=TRUE)
# Classical effects qqplot
qqplot(DanielPlot(m,alpha=0.1)$x,DanielPlot(m)$y)
qqline(DanielPlot(m,alpha=0.1)$y)

# half normal plot of effects
DanielPlot(m,code=TRUE,alpha=0.1,half=TRUE)



In [ ]:
summary(Weight_final_m <- aov(Weight ~ A + B + C + A:B, data=m))
plot(Weight_final_m)

m_reg_Weight <- lm(Weight ~ A + B + C + A:B, data=m)
summary(m_reg_Weight)
op <- par(mfrow=c(2,2))
plot(m_reg_Weight)




In [ ]:
# Viscosity
MEPlot(m, response = "Viscosity")
IAPlot(m, response = "Viscosity")
# vypada to, ze A a B budou mit vliv
# v interakcich toho moc videt neni

summary(aov(Viscosity ~ A*B*C*D, data=m))


In [ ]:
model00 = lm(Viscosity~A*B*C*D,  data=m)
barplot(sort(model00$coeff[2:(2^4-1)]),las = 2)
qqnorm(model00$coeff[2:(2^4-1)],cex = 1.3,pch = 15)
qqline(model00$coeff[2:(2^4-1)],cex = 1.3,pch = 15)

summary(vis_final_m <- aov(Viscosity ~ A+B, data=m))
plot(vis_final_m)


In [ ]:
m_reg_vis <- lm(Viscosity ~ A + B , data=m)
summary(m_reg_vis)
op <- par(mfrow=c(2,2))
plot(m_reg_vis)
# outlier in measurement n. 12?


In [ ]:
# 6.32
new.data <- t(rbind(seq(100,120,by=.5),seq(4,8,by=.1),seq(20,30,by=.25)))
colnames(new.data) <-c("Temperature","Concentration","Time")
new.data <- as.data.frame(new.data)
new.data <- expand.grid(new.data)


In [ ]:

Weight_num_m <- lm(Weight ~ Temperature + Concentration + Time + Temperature:Concentration, data=Problem_631)
new.data$fit  <- predict(Weight_num_m,new.data)
contourplot(fit~Temperature*Concentration,new.data[new.data$Time == 24,],xlab="A Temperature",ylab="B Concentration")
# pro max Weight zvysit Temperature, Concentration i Time


In [ ]:

vis_num_m <- lm(Viscosity ~ Temperature + Concentration, data=Problem_631)
new.data$fit2 <- predict(vis_num_m, new.data)
contourplot(fit2~Temperature*Concentration,new.data[new.data$Time == 25,],xlab="A Temperature",ylab="B Concentration")
# For Viscosity  Temperature and Concentration down

# for Weight between 2400-2500 and min. Viscosity:
# Concentration 4, Temperature 100, Time 20

In [ ]:

p.lm            <- lm(Weight ~ Temperature + Concentration + Time + Temperature:Concentration, data=Problem_631)
p.lm2           <- lm(Viscosity ~ Temperature + Concentration, data=Problem_631)
p.tmp           <- list(Temperature=seq(100,120,by=1),Concentration=seq(4,8,by=.1))
p.new.data      <- expand.grid(p.tmp)
p.new.data$Time <- 24
p.new.data$fit  <- predict(p.lm ,p.new.data)


In [ ]:

p.new.data2  <- p.new.data
p.new.data2$fit  <- predict(p.lm2 ,p.new.data)
p.new.data$response = "Weight"
p.new.data2$response= "Viscosity"
p.data = rbind(p.new.data,p.new.data2)



In [ ]:

p.new.data3 <- p.new.data2 %>% 
  mutate(fit2  = predict(p.lm ,p.new.data)) %>% 
  rename(Viscosity = fit, 
         Weight = fit2) %>% 
  dplyr::select(-response)
head(p.new.data3)



In [ ]:
g <- ggplot(p.new.data3, aes(Temperature, Concentration)) +
  geom_contour(aes(z = Viscosity),col= "red",binwidth = 10) +
  geom_contour(aes(z = Weight), col = "blue",binwidth = 10) +
  geom_label_contour(aes(z = Viscosity), col="red") +
  geom_label_contour(aes(z = Weight),col="blue")
g



In [ ]:
ggplot(p.new.data, aes(Temperature,Concentration)) +
  geom_contour_filled(aes(z = fit)) +
  labs(title = "Contour plot of p",
       x="A:Temperature",y="C:Concentration",
       level = "Weight") 




In [ ]:

p2 = ggplot(p.data, aes(x=Temperature , y=Concentration , z=fit, colour=response )) +
  stat_contour(aes(alpha=..level..), binwidth=10) +
  theme(panel.background=element_rect(fill="white")) +
  theme(panel.grid=element_blank()) +
  labs(title="Plot 2")
p2


In [ ]:

mutate_cond <- function(.data, condition, ..., envir = parent.frame()) {
  condition <- eval(substitute(condition), .data, envir)
  .data[condition, ] <- .data[condition, ] %>% mutate(...)
  .data
}


In [ ]:

p.data2 <- p.data %>% 
  mutate_cond(response == "Weight", fit = fit - 2400) %>% 
  mutate_cond(response == "Viscosity", fit = fit - 1500)


In [ ]:

p3 = ggplot(p.data2, aes(x=Temperature , y=Concentration , z=fit, colour=response )) +
  stat_contour(aes(alpha=..level..), binwidth=10) +
  theme(panel.background=element_rect(fill="white")) +
  theme(panel.grid=element_blank()) +
  labs(title="Plot 2")
p3

